In [1]:
import rdflib
from rdflib.plugins.sparql import prepareQuery
from tabulate import tabulate

In [2]:
filename = "../../data/07/ABox.ttl"

In [3]:
text1 = '''CQ_7.1
What are the steps that are part of the workflow for the risk assessment of the museum? What is their duration? What are the activities they were executed in?
'''

query1 = '''
PREFIX param: <http://www.ontologydesignpatterns.org/cp/owl/parameter.owl#>
PREFIX pwo: <http://purl.org/spar/pwo/>
PREFIX taskex: <http://www.ontologydesignpatterns.org/cp/owl/taskexecution.owl#>
PREFIX time: <http://www.w3.org/2006/time#>

SELECT DISTINCT ?step ?duration_value ?activity
WHERE {
  ?workflow pwo:hasStep ?step .
  ?step param:hasParameter ?duration ;
    taskex:isExecutedIn ?activity .
  ?duration time:days ?duration_value .
}
'''

In [4]:
text2 = '''CQ_7.2
What are the activities involved in the event executing the workflow? What are the time interval in which they respectively are executed?
'''

query2 = '''
PREFIX pwo: <http://purl.org/spar/pwo/>
PREFIX tisit: <http://www.ontologydesignpatterns.org/cp/owl/timeindexedsituation.owl#>
PREFIX ti: <http://www.ontologydesignpatterns.org/cp/owl/timeinterval.owl#>

SELECT DISTINCT ?activity ?start_date ?end_date
WHERE {
    ?workflow_execution pwo:involvesActivity ?activity .
    ?activity tisit:atTime ?time_interval .
    ?time_interval ti:hasIntervalStartDate ?start_date ;
      ti:hasIntervalEndDate ?end_date .
}
'''

In [5]:
text3 = '''CQ_7.3
What does each assessment activity target? What does it assess?
'''

query3 = '''
PREFIX hero: <https://w3id.org/sirius/ontology/hero/>

SELECT ?activity ?heritage_asset ?element
WHERE {
    ?activity hero:targets ?heritage_asset ;
      hero:assesses ?element .
}
'''

In [6]:
text4 = '''CQ_7.4
Who participated in each assessment activity? When? What did it target? What did it assess? What is it documented by?
'''

query4 = '''
PREFIX hero: <https://w3id.org/sirius/ontology/hero/>
PREFIX partic: <http://www.ontologydesignpatterns.org/cp/owl/participation.owl#>
PREFIX tisit: <http://www.ontologydesignpatterns.org/cp/owl/timeindexedsituation.owl#>
PREFIX ti: <http://www.ontologydesignpatterns.org/cp/owl/timeinterval.owl#>


SELECT ?activity ?agent ?start_date ?end_date ?heritage_asset ?element ?source
WHERE {
    ?activity partic:hasParticipant ?agent ;
      tisit:atTime ?time_interval ;
      hero:targets ?heritage_asset ;
      hero:assesses ?element .
    ?time_interval ti:hasIntervalStartDate ?start_date ;
      ti:hasIntervalEndDate ?end_date .
    OPTIONAL {
      ?activity hero:isDocumentedBy ?source .
    }
}
'''

In [7]:
queries = [(text1, query1),
           (text2, query2),
           (text3, query3),
           (text4, query4)
           ]

g = rdflib.ConjunctiveGraph()
g.parse(filename, format="turtle", encoding="utf-8")

for query in queries:
    q = prepareQuery(query[1])
    results = g.query(q)
    print(query[0])
    table = []
    for row in results:
        table.append([row[var] for var in results.vars])
    print(tabulate(table, headers=results.vars, tablefmt="psql"))

CQ_7.1
What are the steps that are part of the workflow for the risk assessment of the museum? What is their duration? What are the activities they were executed in?

+--------------------------------------------------------+------------------+------------------------------------------------------------------+
| step                                                   |   duration_value | activity                                                         |
|--------------------------------------------------------+------------------+------------------------------------------------------------------|
| https://w3id.org/sirius/ontology/data/07/identify-step |               30 | https://w3id.org/sirius/ontology/data/07/identification-activity |
| https://w3id.org/sirius/ontology/data/07/analyze-step  |                5 | https://w3id.org/sirius/ontology/data/07/analysis-activity       |
| https://w3id.org/sirius/ontology/data/07/evaluate-step |               30 | https://w3id.org/sirius/ontolo